In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1680198742267/work


In [8]:
import pickle
import pandas as pd
import statistics

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
statistics.stdev(y_pred)

5.28140448146535

In [18]:
year = 2022
month = 2
output_file = f'fhv_tripdata_{year:04d}_{month:02d}.parquet'

In [19]:
df_preds = pd.DataFrame(y_pred, columns=['Predictions'])
df_preds['ride_id'] = f'{year:04}_{month:02}_' + df_preds.index.astype('str')

In [21]:
df_preds.to_parquet(path=f'data/{output_file}', engine='pyarrow', compression=None, index=False)